In [1]:
import pandas as pd

In [2]:
from google.colab import files


In [3]:
# --- 1) Importer ton fichier CSV depuis ton ordinateur ---
uploaded = files.upload()

Saving gares-de-voyageurs (1).csv to gares-de-voyageurs (1).csv


In [4]:
# Récupérer automatiquement le nom du fichier téléchargé
file_name = list(uploaded.keys())[0]

In [5]:
# --- 2) Charger la base ---
df = pd.read_csv(file_name, sep=";", encoding="utf-8")  # auto-détection du séparateur

df.head()

,Nom,Trigramme,Segment(s) DRG,Position géographique,Code commune,Code(s) UIC
0,Abancourt,ABT,C,"49.6852237, 1.7743058",60001,87313759
1,Abbaretz,AAR,C,"47.5546432, -1.5244159",44001,87481614
2,Abbeville,ABB,B,"50.10221, 1.82449",80001,87317362
3,Ablon-sur-Seine,ABL,B,"48.725468, 2.419151",94001,87545269
4,Achères Grand Cormier,GCR,B,"48.9551835, 2.0919031",78551,87386052


In [6]:
df.head()
df.columns


Index(['Nom', 'Trigramme', 'Segment(s) DRG', 'Position géographique',
       'Code commune', 'Code(s) UIC'],
      dtype='object')

In [7]:
df.columns = df.columns.str.strip().str.replace("\ufeff", "")  # enlève BOM + espaces
df.columns


Index(['Nom', 'Trigramme', 'Segment(s) DRG', 'Position géographique',
       'Code commune', 'Code(s) UIC'],
      dtype='object')

In [8]:
df.head()


,Nom,Trigramme,Segment(s) DRG,Position géographique,Code commune,Code(s) UIC
0,Abancourt,ABT,C,"49.6852237, 1.7743058",60001,87313759
1,Abbaretz,AAR,C,"47.5546432, -1.5244159",44001,87481614
2,Abbeville,ABB,B,"50.10221, 1.82449",80001,87317362
3,Ablon-sur-Seine,ABL,B,"48.725468, 2.419151",94001,87545269
4,Achères Grand Cormier,GCR,B,"48.9551835, 2.0919031",78551,87386052


In [9]:
# --- 2) Liste des villes + département correspondant ---
villes_departements = {
    # Bouches-du-Rhône (13)
    "Arles": "Bouches-du-Rhône",
    "Martigues": "Bouches-du-Rhône",
    "La Ciotat": "Bouches-du-Rhône",
    "Aubagne": "Bouches-du-Rhône",
    "Cassis": "Bouches-du-Rhône",
    "Carry-le-Rouet": "Bouches-du-Rhône",
    "Sausset-les-Pins": "Bouches-du-Rhône",
    "Ensuès": "Bouches-du-Rhône",

    # Var (83)
    "Sanary-sur-Mer": "Var",
    "Bandol": "Var",
    "Saint-Raphaël": "Var",
    "Fréjus": "Var",
    "Hyères": "Var",
    "Toulon": "Var",
    "Les Arcs": "Var",
    "Vidauban": "Var",
    "Le Luc": "Var",

    # Vaucluse (84)
    "Avignon": "Vaucluse",
    "Orange": "Vaucluse",
    "L’Isle-sur-la-Sorgue": "Vaucluse",
    "Cavaillon": "Vaucluse",
    "Carpentras": "Vaucluse",

    # Alpes-de-Haute-Provence (04)
    "Manosque": "Alpes-de-Haute-Provence",
    "Oraison": "Alpes-de-Haute-Provence",
    "La Brillanne": "Alpes-de-Haute-Provence",
    "Volx": "Alpes-de-Haute-Provence",
    "Sisteron": "Alpes-de-Haute-Provence",

    # Hautes-Alpes (05)
    "Gap": "Hautes-Alpes",
    "Briançon": "Hautes-Alpes",
    "Embrun": "Hautes-Alpes",
    "Veynes": "Hautes-Alpes",
    "Montdauphin": "Hautes-Alpes",

    # Alpes-Maritimes (06)
    "Cannes": "Alpes-Maritimes",
    "Antibes": "Alpes-Maritimes",
    "Menton": "Alpes-Maritimes",
    "Grasse": "Alpes-Maritimes",
    "Villefranche-sur-Mer": "Alpes-Maritimes",
    "Tende": "Alpes-Maritimes",
    "Saorge": "Alpes-Maritimes",
    "Breil-sur-Roya": "Alpes-Maritimes"
}

# --- 3) Filtrer les lignes dont la colonne 'Nom' contient l’une de ces villes ---
pattern = "|".join(villes_departements.keys())
df_filtre = df[df["Nom"].str.contains(pattern, case=False, na=False)]

# --- 4) Ajouter la colonne "Département" ---
def get_departement(nom_gare):
    for ville, dep in villes_departements.items():
        if ville.lower() in nom_gare.lower():
            return dep
    return None

df_filtre["Département"] = df_filtre["Nom"].apply(get_departement)

# --- 5) Afficher le résultat ---
df_filtre.head()

# --- 6) Enregistrer la nouvelle base filtrée ---
df_filtre.to_csv("gares_filtrees_paca.csv", index=False)
files.download("gares_filtrees_paca.csv")

/tmp/ipython-input-3360115074.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtre["Département"] = df_filtre["Nom"].apply(get_departement)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
!pip install folium


In [17]:
import pandas as pd
from google.colab import files

print("⬆️ Upload le CSV des gares PACA")
uploaded_gares = files.upload()
gares_file = list(uploaded_gares.keys())[0]

print("⬆️ Upload le CSV des horaires d'ouverture")
uploaded_horaires = files.upload()
horaires_file = list(uploaded_horaires.keys())[0]

df_gares = pd.read_csv(gares_file, encoding="utf-8")
df_horaires = pd.read_csv(horaires_file, encoding="utf-8")


⬆️ Upload le CSV des gares PACA


Saving gares_filtrees_paca (1).csv to gares_filtrees_paca (1) (1).csv
⬆️ Upload le CSV des horaires d'ouverture


Saving horaires-des-gares1 (1).csv to horaires-des-gares1 (1).csv


In [35]:
df_horaires["UIC"] = df_horaires["UIC"].astype(int)

df_horaires_filtre = df_horaires[
    df_horaires["UIC"].isin(df_gares["UIC"])
]


In [36]:
df_final = df_horaires_filtre.merge(
    df_gares,
    on="UIC",
    how="left"
)


In [37]:
df_final.columns


Index(['UIC', 'Gare', 'Jour de la semaine', 'Horaire en jour normal',
       'Horaire en jour férié', 'Nom', 'Trigramme', 'Segment(s) DRG',
       'Position géographique', 'Code commune', 'Département'],
      dtype='object')

In [38]:
df_final = df_final[[
    "Nom",
    "Département",
    "Code commune",
    "Position géographique",
    "UIC",
    "Jour de la semaine",
    "Horaire en jour normal",
    "Horaire en jour férié"
]]


In [39]:
ordre_jours = [
    "Lundi", "Mardi", "Mercredi",
    "Jeudi", "Vendredi", "Samedi", "Dimanche"
]

df_final["Jour de la semaine"] = pd.Categorical(
    df_final["Jour de la semaine"],
    categories=ordre_jours,
    ordered=True
)

df_final = df_final.sort_values(
    by=["Nom", "Jour de la semaine"]
)


In [40]:
df_final.to_csv("gares_PACA_horaires_geo.csv", index=False)
df_final.head(10)


,Nom,Département,Code commune,Position géographique,UIC,Jour de la semaine,Horaire en jour normal,Horaire en jour férié
168,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Lundi,05:10-23:30,05:10-00:30
81,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Mardi,05:10-23:30,05:10-00:30
169,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Mercredi,05:10-23:30,05:10-00:30
75,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Jeudi,05:10-23:30,05:10-00:30
200,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Vendredi,05:10-23:30,05:10-00:30
16,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Samedi,05:40-00:30,05:10-00:30
91,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Dimanche,05:10-00:30,05:10-00:30
216,Arles,Bouches-du-Rhône,13004,"43.684853, 4.63203",87753657,Lundi,05:45-22:30,06:45-22:30
156,Arles,Bouches-du-Rhône,13004,"43.684853, 4.63203",87753657,Mardi,05:45-22:30,06:45-22:30
57,Arles,Bouches-du-Rhône,13004,"43.684853, 4.63203",87753657,Mercredi,05:45-22:30,06:45-22:30


In [46]:
df_final = df_final[df_final["UIC"] != 87271494]


In [47]:
df_final.to_csv("gares_PACA_horaires_geo_clean.csv", index=False)


In [48]:
# Séparer latitude et longitude
df_final[["Latitude", "Longitude"]] = df_final["Position géographique"].str.split(
    ",", expand=True
)

# Convertir en float
df_final["Latitude"] = df_final["Latitude"].astype(float)
df_final["Longitude"] = df_final["Longitude"].astype(float)

df_final.head()


,Nom,Département,Code commune,Position géographique,UIC,Jour de la semaine,Horaire en jour normal,Horaire en jour férié,Latitude,Longitude
168,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Lundi,05:10-23:30,05:10-00:30,43.585784,7.119498
81,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Mardi,05:10-23:30,05:10-00:30,43.585784,7.119498
169,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Mercredi,05:10-23:30,05:10-00:30,43.585784,7.119498
75,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Jeudi,05:10-23:30,05:10-00:30,43.585784,7.119498
200,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Vendredi,05:10-23:30,05:10-00:30,43.585784,7.119498


In [49]:
import folium

# Centre approximatif PACA
map_paca = folium.Map(
    location=[43.7, 6.0],
    zoom_start=7,
    tiles="OpenStreetMap"
)


In [50]:
df_gares_map = df_final.drop_duplicates(subset=["UIC"])


In [51]:
for _, row in df_gares_map.iterrows():
    popup_text = f"""
    <b>{row['Nom']}</b><br>
    Département : {row['Département']}<br>
    UIC : {row['UIC']}
    """

    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=popup_text,
        icon=folium.Icon(icon="train", prefix="fa")
    ).add_to(map_paca)


In [52]:
map_paca
